In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj02.ipynb")

In [ ]:
from utils import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-muted")
import statsmodels.api as sm

# Project 2: Mariel Boatlift
In this project, we will make an attempt to replicate a famous paper "_The Impact of the Mariel Boatlift on the Miami Labor Market_" by Prof. David Card on the effects of the Mariel Boatlift on the Miami labor market. 

#### Historical Background
Amidst an economic downturn in Cuba and an increasing number of dissident Cubans seeking asylum, the Cuban leader Fidel Castro announced on April 20, 1980 that any Cuban who wished to leave the island could do so, reversing the Communist regime’s closed emigration policy. Between April and October 1980, some 1,700 boats, many arranged by Cuban exiles already in the United States, carried Cubans from the port of Mariel (the departure zone designated by the Castro government) to Florida. 

In response, President Jimmy Carter declared  a state of emergency in affected areas and, on June 20, established the the Cuban-Haitian Entrant Program (CHEP), which granted temporary status and access to asylum processing and community assistance to both Cubans and thousands of Haitians concurrently fleeing to the United States. During this period, reports that the Cuban government also released prisoners to travel to the United States prompted the U.S. Coast Guard to blockade some 1,400 boats; however, hundreds of Cubans continued to arrive in Florida daily. The CHEP coverage window ended after the Carter administration negotiated an end to the boat lifts with the Cuban government in October 1980. ([Source](https://immigrationhistory.org/item/mariel-boatlift/))

#### Economic Impact
About half of the Mariel immigrants decided to live in Miami permanently, which resulted in "a 7\% increase in the labor force of Miami and a 20\% increase in the number of Cuban workers in Miami" ([Card (1990)](https://davidcard.berkeley.edu/papers/mariel-impact.pdf)). Aside from the unemployment rate rising from 5.0\% in April 1980 to 7.1\% in July, the actual damage to the economy was marginal and followed trends across the United States at the time. When observing data from 1979 to 1985 on the Miami labor market and comparing it with similar data from several other major cities across the United States, focusing on wages, the effects of the boatlift were marginal ([Alejandro and Leif (1989)](https://www.jstor.org/stable/2095716)). 

#### Research Approach
Since immigrants have the freedom to choose their destinations (and will naturally choose those places with strong labor markets), it is not enough to look at whether places with more immigrants and see if these places have different unemployment rates or wages to determine the causal effect of immigrantion on a local labor market. However, Card determined that there are some circumstances under which immigrants will arrive which have very little to do with the labor market of that place; the Mariel Boatlift is one such event. Here is Card's description:

> The experiences of the Miami labor market in the aftermath of the Mariel Boatlift form one such \["natural"\] experiment. From May to September 1980, some 125,000 Cuban immigrants arrived in Miami on a flotilla of privately chartered boats. Their arrival was the consequence of an unlikely sequence of events culminating in Castro's declaration on April 20, 1980, that Cubans wishing to emigrate to the United States were free to leave from the port of Mariel. Fifty percent of the Mariel immigrants settled permanently in Miami. The result was a 7% increase in the labor force of Miami and a 20% increase in the number of Cuban workers in Miami. (Card, 1990:245-6)

Professor David Card received the Nobel Prize in Economics "for his empirical contributions to labour economics" which is highly relevant to this work. He delivered his [prize lecture](https://youtu.be/wD48p6m8U-8?t=237) on 8 December 2021.

**Reading:** You should download a copy of the original paper and read at least pages 245-251 and pages 255-257. It is available [here](http://davidcard.berkeley.edu/papers/mariel-impact.pdf).

### References

This notebook is based on another assignment by [Prof. Josh Goldstein](https://courses.demog.berkeley.edu/goldstein175).

**A Note on Grading:**  
In Project 2, the autograded questions will have hidden tests, and the text-based free response questions will be graded on correctness. 

---
## Part 1: CPS Data

First, we will load in data from the Current Population Survey (CPS). The CPS "outgoing rotation groups" that we are using for the analysis is the largest sample available for this time period. Still, once we limit ourselves to Miami and the comparison cities, the sample sizes are still small. 

In [ ]:
mariel_raw = pd.read_csv("mariel-boatlift.csv")
mariel_raw.head(5)

The columns of our table have some odd names. We provide descriptions of the variables of interest below.

| Column Name | Description |
|-----|-----|
| age | Age of individual |
| smsarank | City |
| esr | Employment status |
| ftpt79 | Full-time or part-time |
| earnhre | Nominal hourly pay in cents |
| educ | Level of education (BA, HS diploma, or < HS) |
| ethrace | Race & ethnicity |

Before moving onto the analysis of the data, there is a tiny bit of data cleaning that needs to be done. The data currently have a `.id` column, which encodes the year. However, these values range from 1 to 7, so to get the actual years, we need to add 1978 to each.

**Question 1.1:** Add 1978 to each value in the `.id` column of `mariel_raw`. Create a new table `mariel` with the same columns as `mariel_raw` but with a `year` column and no `.id` column.

The resulting `mariel` dataframe should look like the following:

| age | smsarank | esr | ftpt79 | earnhre | educ | ethrace | year |
| ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- |
| 71 | Los Angeles | Housework | Not In Labor Force | NaN | lessHS | whites | 1979 |
|  |  |  |  |  |  |  | ... (rows omitted) |

In [ ]:
mariel = mariel_raw.copy()
...
mariel.head()

In [ ]:
grader.check("q1_1")

Because we are focusing on certain racial groups in this lab, we want to remove the rows of `mariel` that are not in the set of ethnicities that we are concerned with.

**Question 1.2:** Remove the rows of `mariel` where the `ethrace` variable is `"others"`.

In [ ]:
mariel = ...
mariel.head(5)

In [ ]:
grader.check("q1_2")

**Question 1.3:** What categories are there in the `ethrace` variable (now)?

1. White and Black
2. White and Black and Cuban
3. White, Black, Cuban, and Hispanic
4. Cuban, Hispanic, non-Hispanic

Assign the number corresponding to your answer to `q1_3` below.

In [ ]:
q1_3 = ...

In [ ]:
grader.check("q1_3")

**Question 1.4:** What are the units of the `earnhre` variable? Read the relevant CPS data [documentation](https://data.nber.org/morg/docs/cpsx.pdf).   

Hint: Use `Cmd+F` or `Ctrl+F`. 

1. Cents per hour
2. Dollars per hour
3. Cents per day
4. Dollars per day
5. Cents per week
6. Dollars per week

Assign the number corresponding to your answer to `q1_4` below.

In [ ]:
q1_4 = ...

In [ ]:
grader.check("q1_4")

**Question 1.5:** Which cities can make up the ideal comparison group in this study?

1. All U.S. cities except Miami
2. All Florida cities except Miami
3. Cities nearby that are subject to the same macroeconomic influences as Miami but didn't receive many Cuban immigrants.
4. Cities nearby that are subject to the same macroeconomic influences as Miami but received many other immigrants.

Assign the number corresponding to your answer to `q1_5` below.

In [ ]:
q1_5 = ...

In [ ]:
grader.check("q1_5")

---
## Part 2: Unemployment 

We're going to begin by trying to replicate Card's results for unemployment in his Table 4. (We'll do wages in Table 3 later).

In the cell below, we define a function `get_ue` that takes an array of values from the `esr` column and returns the proportion of people who are unemployed (the unemployment rate).

In [ ]:
def get_ue(esr):
    unemployed_looking = sum(esr == "Unemployed-Looking")
    others = sum(np.isin(esr, np.array(["Unemployed-Looking",  "Employed-At Work", "Employed-Absent"])))
    return unemployed_looking / others

When considering the effect on the unemployment rate of the boatlift, we want to first separate Miami from the comparison cities and select for the desired age group (16 to 61). After we have the desired rows, we want to create a table where the rows represent a year in the data, the columns represent the unique values of the `ethrace` variable, and the cells contain the unemployment rate.



In [ ]:
miami_ue = mariel[(mariel["smsarank"] == "Miami") & \
                  (mariel["age"] >= 16) & \
                  (mariel["age"] <= 61)]

**Question 2.1:** Using `miami_ue`, create a lineplot that shows the unemployment rate for Miami from 1979 to 1985. Label your plot properly (both axes and title). 
Hint: You may want to use `groupby` with `get_ue` (defined above) as the aggregation function. 

In [ ]:
def plot_miami_ue():
    miami_ue_by_year = ...
    miami_ue_by_year = miami_ue_by_year.reset_index()

    plt.vlines(1980, 0, 0.2, color="grey", linestyle="--", linewidth=2)
    plt.plot(...) # feel free to also change the style like color, linewidth, etc
    ...
    
    return miami_ue_by_year
miami_ue_by_year = plot_miami_ue()

In [ ]:
grader.check("q2_1")

Now we want to explore unemployment statistics by ethnicity. This is accomplished by creating a pivot table (using `pd.pivot_table()`). The `.pivot_table()` method can also take as an argument (`aggfunc`) an aggregator function, to which it will pass an array of values that corresponding to each row-column pair. In this case, it will pass the array of `esr` values that correspond to each `year`-`ethrace` pair to the function we pass, which will be the `get_ue` function define above. You will need to pass one of `year` and `ethrace` as the `index`, with the other one serving as the `columns`.

The end result is that we will have a table where each column is an `ethrace` value, each row is a year, and the values are the unemployment rate for that `ethrace` value. All of this is done for you in the cell below, and the results are stored as `miami_ue_ethrace`.

In [ ]:
miami_ue_ethrace = pd.pivot_table(miami_ue, values='esr', index=['year'], columns=['ethrace'], aggfunc=get_ue)
miami_ue_ethrace

**Question 2.2:** Create the same pivot table below (include the same age restriction), but for the comparison cities (that is, all cities _except for Miami_). **Because we are looking at the comparison cities, we must exclude Cubans using a filter on the `ehtrace` columns.** Store the pivot table as `not_miami_ue_ethrace`.

In [ ]:
not_miami_ue = ... # select relevant rows
not_miami_ue_ethrace = ... # pivot table
not_miami_ue_ethrace

In [ ]:
grader.check("q2_2")

In the cell below, we plot the unemployment rates for Miami and the comparison cities for each `ethrace` value. The dashed vertical line in 1980 indicates the Mariel boatlift's occurance. (The function `plot_ue_by_ethrace`, along with the other plotting functions in this notebook, are defined in the `utils.py` file if you want to look at them. They're hidden because the code is very verbose.)

In [ ]:
plot_ue_by_ethrace(miami_ue_ethrace, not_miami_ue_ethrace)

<!-- BEGIN QUESTION -->

**Question 2.3:** Why do the "Cubans" have no comparison group? Explain in the context of the study. 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.4:** Unemployment after the Mariel boatlift goes up for all groups, rising from 5.0% in April 1980 to 7.1% in July. Why does Card argue that "there is no evidence that the Mariel influx adversely affected the unemployment rate of either whites or blacks" (p. 250)?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 2.5:** How much attention should we pay to the ups and downs in these graphs? Are these chance fluctuations from the sample survey ("noise"), or are they important information that we should pay attention to ("signal")?

1. The fluctuations doesn't seem arbitrary, so they are signal
2. The trend reverses very quickly and very suddenly, so they are just noises
3. We can’t tell just by looking, but one could in theory (and with the help of a statistics course) quantify the magnitude of fluctuations that we would expect from random sampling.

Assign the number corresponding to your answer to `q2_4` below.

In [ ]:
q2_5 = ...

In [ ]:
grader.check("q2_5")

---
## Part 3: Wages

Now we will try to replicate Card's findings that the Mariel boatlift also had little or no effect on wages of natives. For simplicity we will not deflate the wages but instead consider the nominal wages.

Because some of the values in the `earnhre` column are missing (`nan`), we remove the rows where this is the case in the cell below. **Throughout this part, make sure you use `mariel_ehre` instead of `mariel`, or else your calculations may error.**

In [ ]:
mariel_ehre = mariel[np.isnan(mariel["earnhre"]) == False].copy()

In order to make the wages more linear and to put them on an easier-to-understand scale, we take the natural log of each value in the `earnhre` column and store this as `log_w`.

In [ ]:
log_w = np.log(mariel_ehre["earnhre"]/100)
mariel_ehre["log_w"] = log_w
mariel_ehre.head(5)

We want to create a similar pivot table as in part 2, except we want the values in this table to be the mean of the log of wages. We create this table for Miami below, making sure to also filter `merial_ehre` for rows where the individual is employed full-time.

In [ ]:
miami_wages = mariel_ehre[(mariel_ehre["age"] >= 16) & \
                          (mariel_ehre["age"] <= 61) & \
                          (mariel_ehre["smsarank"] == "Miami") & \
                          (mariel_ehre["ftpt79"] == "Employed full-time")]
miami_wages_ethrace = pd.pivot_table(miami_wages, values="log_w", index=["year"], columns=["ethrace"], aggfunc=np.mean)
miami_wages_ethrace

**Question 3.1:** Create the same pivot table below, except for the comparison cities (that is, all cities _except for Miami_). Store the pivot table as `not_miami_wages_ethrace`.

In [ ]:
not_miami_wages = ... # select relevant rows
not_miami_wages_ethrace = ... # pivot table
not_miami_wages_ethrace

In [ ]:
grader.check("q3_1")

In the cell below, we plot the wages for Miami and the comparison cities for each `ethrace` value.

In [ ]:
plot_wages_by_ethrace(miami_wages_ethrace, not_miami_wages_ethrace)

Our numbers differ from Card's Table 4 because we are not accounting for inflation. In order to make inferences about the effect of the boatlift on wages easier, let's plot the differences between Miami and the Comparison Cities. (The difference being plotted here is log wages for Miami *minus* log wages for comparison cities)

In [ ]:
plot_wage_diffs_by_ethrace(miami_wages_ethrace, not_miami_wages_ethrace)

**Question 3.2:** If wages were hurt by the influx of migrants, we would expect this graph to show

1. A decrease after 1980, as Miami wages went down relative to other cities
2. Values below 0 for all periods, because Miami would always have lower wages
3. An uptick after 1980 because we are working with logarithms.

Assign the number corresponding to your answer to `q3_2` below.

_Hint:_ $\log A  - \log B = \log \frac{A}{B}$

In [ ]:
q3_2 = ...

In [ ]:
grader.check("q3_2")

So it seems that indeed our analysis is consistent with Card's conclusion that "the Mariel immigration had virtually no effect on wages or unemployment outcomes of non-Cuban workers in the Miami labor market" (p. 255).

---
## Part 4: Education

We would expect any negative effect of the influx of immigrants to be strongest on the group that they most resemble. Because most of the Cuban immigrants in the boatlift were unskilled, we would expect the strongest effect on natives with the least education, with perhaps the clearest comparison group being Hispanics with the least education.

Card used a different approach, looking at the effects for low-skilled workers by predicting wages based on education and years of experience. Here we do something a bit simpler, using education only.

<!-- BEGIN QUESTION -->

**Question 4.1:** If the boatlift had a negative effect on the employment of unskilled workers, what would we expect to see in the unemployment for each of categories of education in both Miami and the comparison cities? 

_Note:_ The possible values of `educ` are `BA`, `HS`, or `lessHS`.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

We want to create a similar pivot table as in parts 2 and 3, except we want the values in this table to be the mean of the unemployment rate partitioned by _education_, not `ethrace`. We create this table for Miami below.

In [ ]:
miami_educ_ue = mariel[(mariel["age"] >= 16) & \
                       (mariel["age"] <= 61) & \
                       (mariel["smsarank"] == "Miami")]
miami_educ_ue = pd.pivot_table(miami_educ_ue, values="esr", index=["year"], \
                               columns=["educ"], aggfunc=get_ue)
miami_educ_ue

<!-- BEGIN QUESTION -->

**Question 4.2:** What happens to the unemployment rates of those with a college education (BA) between 1980 and 1982, when the effects of the Mariel boatlift should have been felt? What happens to those with the least education? ("lessHS"). Is this consistent with a large effect of immigration on the least educated that is hypothesized above?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 4.3:** Create the same pivot table below, except for the comparison cities (that is, all cities _except for Miami_). Store the pivot table as `not_miami_educ_ue`.

In [ ]:
not_miami_educ_ue = ... # select relevant rows
not_miami_educ_ue = ... # pivot table
not_miami_educ_ue

In [ ]:
grader.check("q4_3")

In the cell below, we plot the unemployment rates for Miami and the comparison cities for each `educ` value.

In [ ]:
plot_ue_by_educ(miami_educ_ue, not_miami_educ_ue)

<!-- BEGIN QUESTION -->

**Question 4.4:** Like Card's study, many empirical works find very small or no impact of immigration on local workers' wages and employment. Several studies even found positive impact of skilled immigration on wages and employment. What are 2 possible reasons that having immigrants would benefit the native-born workers?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

---
## Part 5: What Determines Wages?
Now we want to use the data and expand on the above analysis by asking what variables are related to wages. We will use the basic OLS regression to quantitatively determine the association. This is also part of the analysis Card used to produce Table 7 in his paper. 

**Question 5.1:** Filter `mariel_ehre` for individuals that are being employed full-time. Assign the result to `mariel_full_time`. 

In [ ]:
mariel_full_time = ...
mariel_full_time

In [ ]:
grader.check("q5_1")

What variables in our dataframe are related to wages? It seems like `educ`, `ethrace`, `year`, and `smsarank` can have an influence on individuals' income. 

But note that these columns are strings, and we cannot run regression on string-values columns. To get around this, we will create some dummy variables (some refers to this process as "one-hot encoding"). 

For example, let's create some dummies for `educ` using `pd.get_dummies`. 

In [ ]:
educ_dummies = pd.get_dummies(mariel_full_time["educ"])
educ_dummies

And we can add them back to a dataframe using `join`. But note that we should not include all the dummy columns, otherwise we will run into multicollinearity (which is bad). This is called the "dummy variable trap". So in our case, we will exclude the last column in dummy variables for each variable. 

In [ ]:
# Let's do everything all at once
mariel_regression = mariel_full_time.copy() # copy the dataframe
educ_dummies = pd.get_dummies(mariel_full_time["educ"]) # create dummies for educ
educ_dummies = educ_dummies.iloc[:, :-1] # exclude the last column
mariel_regression = mariel_regression.join(educ_dummies) # add the dummy vars to dataframe
mariel_regression = mariel_regression.drop(["educ"], axis=1) # drop the original column
mariel_regression

**So how to interpret the dummy variables for education?**

We notice that there are two dummy variable columns for education: `BA` and `HS`. When `BA` is 1, it means the individual in this row has a Bachelor's degree. When `HS` is 1, it means the individual in this row has a high school degree (but not a Bachelor's degree). When both `BA` and `HS` are 0, that implies `lessHS` should be 1, and it means the individual in this rows has less education than a high school degree. 

Therefore when `BA` and `HS` show up in a regression, we can interpret their coefficients as the following:
- Coefficient for `BA`: the average amount of wages an individual with Bachelor's degree earns *minus* the average amount of wages an individual with less than high school degree earns. In other word, if the coefficient is 100, then it means: on average an individual with Bachelor's degree earns 100 more than an individual with less than high school degree. 
- Coefficient for `HS`: the average amount of wages an individual with high school degree earns *minus* the average amount of wages an individual with less than high school degree earns. Example omitted. 

So in short, the coefficients for education (`BA` and `HS`) represent the additional wages earned compared to the base case scenario (which is the column that we excluded, and in this case `lessHS`). 

Okay let's run this simple regression now as an example. 

In [ ]:
X = mariel_regression.iloc[:, -2:] # BA and HS
X = sm.add_constant(X) # add a constant term
y = mariel_regression["earnhre"]
model_1 = sm.OLS(y, X).fit()
model_1.summary()

In [ ]:
# sanity check: 
# the coefficient for constant should be 575.3880
assert np.isclose(model_1.params["const"], 575.3880)

**What is the interpretation of this regression result?**

The results show that on average individuals with less than high school degree education earns 575.39 (unit omitted; you should include it!); on average individuals with high school degree earns 714.4018 (575.3880 + 139.0138); and finally individuals with Bachelor's degree earns 905.2974 (575.3880 + 329.9094) on average. And the result is highly significant given the t-stat and p-values. So in our data, education plays an important role in determining wages. 

Now we will add the remaining variables `ethrace`, `year`, and `smsarank` to our regression. First, generate dummy variables for all of them. Let's start fresh! 

**Question 5.2:** Generate dummy variables for `educ`, `ethrace`, `year`, and `smsarank`. Add those dummy variables to `mariel_regression` and then drop the original column. 

Hint: Be sure to exclude the last dummies for each variable to avoid the "dummy variable trap"!

In [ ]:
mariel_regression = mariel_full_time.copy()
...
mariel_regression

In [ ]:
grader.check("q5_2")

Great job! Now we have our dataframe is ready for regression!

First, let's run an OLS regression using only `educ` and `ethrace`, so that we can get a sense of how these two variables influenced wages (use `earnhre` for now) during the sample period. 

**Question 5.3:** Run a regression with `earnhre` as the explained variable (y) and the dummies for `educ` and `ethrace` as the explanatory variables (X). Include a constant term in your regression. Assign your OLS model to `model_2`. 

In [ ]:
...
model_2.summary()

In [ ]:
grader.check("q5_3")

**Question 5.4:** According to the regression output, how much **more** on average does a Cuban individual with a high school degree earn than another Cuban individual with less than high school education? Assign the number corresponding to your answer to `q5_4` below.

In [ ]:
q5_4 = ...
q5_4

In [ ]:
grader.check("q5_4")

**Question 5.5:** According to the regression output, how much **more** on average does a Cuban individual with a high school degree earn than another white individual with a high school degree? If you think the Cuban individual earns less, put a negative number. Assign the number corresponding to your answer to `q5_5` below.

In [ ]:
q5_5 = ...
q5_5

In [ ]:
grader.check("q5_5")

**Question 5.6:** According to the regression output, how much does a Cuban individual with a high school degree earn on average? Assign the number corresponding to your answer to `q5_6` below.

In [ ]:
q5_6 = ...
q5_6

In [ ]:
grader.check("q5_6")

Now, let's run an OLS regression using all the variables that we have. 

**Question 5.7:** Run a regression with `earnhre` as the explained variable (y) and the dummies for `educ`, `ethrace`, `year` and `smsarank` as the explanatory variables (X). Include a constant term in your regression. Assign your OLS model to `model_3`. 

In [ ]:
...
model_3.summary()

In [ ]:
grader.check("q5_7")

**Question 5.8:** According to the regression output, does the nominal wage (`earnhre`) increase over time in the sample period? Assign the either `True` or `False` to `q5_8` below.

In [ ]:
q5_8 = ...

In [ ]:
grader.check("q5_8")

**Question 5.9:** According to the regression output, which city in the data had the lowest wages on average assume all other variables equal? Assign the `smsarank` of that city to `q5_9` below.

In [ ]:
q5_9 = ...

In [ ]:
grader.check("q5_9")

## Conclusions

In this project, we saw that the Mariel boatlift had little effect on unemployment among ethnic groups in Miami, as the cities that were not receiving \(many\) immigrants were afflicted by the same trends in unemployment. We also saw that the same is true of education-level groups. In fact, many studies find that influxes of immigrants benefit the native-born workers in a region; that they act as complements rather than supplements to the native workforce.

**Congratulations!** You're done with Project 2!

---
## Feedback

**Question 6:** Please fill out this short [feedback form](https://forms.gle/bhMnHpNDYAJPktUn8) to let us know your thoughts about this lab! We really appreciate your opinions and feedback! At the end of the Google form, you should see a codeword. Assign the codeword to the variable `codeword` below. 

In [ ]:
codeword = ...

In [ ]:
grader.check("q6")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()